In [2]:
%reload_ext autoreload
%autoreload 2

import cv2

import os
import sys
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *

from collections import defaultdict

from matplotlib.path import Path
%matplotlib inline

from enum import Enum

class ProposalType(Enum):
    GLOBAL = 'global'
    LOCAL = 'local'
    FREEFORM = 'freeform'
    ALGORITHM = 'algorithm'
    
class PolygonType(Enum):
    CLOSED = 'closed'
    OPEN = 'open'
    TEXTURE = 'textured'
    TEXTURE_WITH_CONTOUR = 'texture with contour'
    DIRECTION = 'directionality'

In [3]:
first, last = section_range_lookup['MD589']
dms = dict([(sc, DataManager(stack='MD589', section=sc, segm_params_id='tSLIC200', load_mask=False)) 
            for sc in range(first, last)])

In [4]:
from FeatureExtractor import *

In [5]:
all_patches_arr = np.load('test.npy')

In [7]:
model_dir = os.environ['MODEL_DIR']+'/vgg16'

fe = FeatureExtractor(model_dir, batch_size=16, ctx='gpu')

# MXNetError: [00:18:25] src/storage/./gpu_device_storage.h:39: Check failed: e == cudaSuccess || e == cudaErrorCudartUnloading CUDA: out of memory
# https://github.com/dmlc/mxnet/issues/675
# changing batch_size does not help
# Restaring notebook, this error is no more !!!

images = all_patches_arr.transpose(0, 3, 1, 2)
print images.shape

features = fe.extract(images)
print features.shape
# np.save(os.path.join(data_dir, '%s_features.npy' % name), features)

(34, 3, 224, 224)
(34, 4096)


In [8]:
label1 = '12N'
label2 = 'Gr'

In [9]:
coef = np.load('%s_%s_linear_svm_coef.npy' % (label1, label2))
intercept = np.load('%s_%s_linear_svm_intercept.npy' % (label1, label2))
selected_feature_indices = np.load('%s_%s_linear_svm_selectedFeatureIndices.npy' % (label1, label2))

In [10]:
y = np.dot(features[:, selected_feature_indices], np.squeeze(coef)) + intercept
Y = np.sign(y)

In [11]:
Y

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1., -1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])